In [1]:
import QuantLib as ql
import openpyxl as xl

In [2]:
def DatetimeToDate(ADatetime):
    ADate = ql.Date(ADatetime.day, ADatetime.month, ADatetime.year)
    return ADate

In [3]:
PstBook = xl.load_workbook('Pst_TBond.xlsx', data_only=True)
MktSheet = PstBook.worksheets[0]
CCY = 'TWD'
Type = 'TBond'
MTMDate = MktSheet.cell(1, 3).value
RW = MktSheet.cell(2, 8).value
settings = ql.Settings.instance()
evDate = DatetimeToDate(MTMDate)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
DC365 = ql.Actual365Fixed()
BDC = ql.Unadjusted
settlementDays = 0

In [4]:
ZeroSP = MktSheet.cell(15, 12).value
Zero6M = MktSheet.cell(17, 12).value
Zero1Y = MktSheet.cell(18, 12).value
Zero3Y = MktSheet.cell(20, 12).value
Zero5Y = MktSheet.cell(21, 12).value
Zero10Y = MktSheet.cell(22, 12).value
Zero30Y = MktSheet.cell(25, 12).value
print(ZeroSP)

0.001861857532461489


In [5]:
DateSP = evDate
Date6M = Cal.advance(evDate, 6, ql.Months, BDC, False)
Date1Y = Cal.advance(evDate, 1, ql.Years, BDC, False)
Date3Y = Cal.advance(evDate, 3, ql.Years, BDC, False)
Date5Y = Cal.advance(evDate, 5, ql.Years, BDC, False)
Date10Y = Cal.advance(evDate, 10, ql.Years, BDC, False)
Date30Y = Cal.advance(evDate, 30, ql.Years, BDC, False)
print(DateSP)

June 30th, 2021


In [6]:
ListDate = list([DateSP, Date6M, Date1Y, Date3Y, Date5Y, Date10Y, Date30Y])
ListRate = list([ZeroSP, Zero6M, Zero1Y, Zero3Y, Zero5Y, Zero10Y, Zero30Y])
VecDate = ql.DateVector(ListDate)
VecRate = ql.DoubleVector(ListRate)
TSZero = ql.ZeroCurve(VecDate, VecRate, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)

In [7]:
TBondSheet = PstBook['TBond']
start_row = 2
end_row = 4
start_col = 0
end_col = 5
recordcount = end_col - start_col + 1
ListTBond = list()
for i in range(start_row, end_row + 1):
    row = [cell.value for cell in TBondSheet[i][start_col:end_col+1]]
    ListTBond.append(row)
    print(row)

['A10105', datetime.datetime(2021, 5, 21, 0, 0), datetime.datetime(2041, 5, 21, 0, 0), 20, 0.00625, 10000000]
['A10106', datetime.datetime(2021, 6, 18, 0, 0), datetime.datetime(2023, 6, 18, 0, 0), 2, 0.00125, 20000000]
['A10107', datetime.datetime(2021, 6, 23, 0, 0), datetime.datetime(2031, 6, 23, 0, 0), 10, 0.00375, 50000000]


In [8]:
TBondBook = list()
for i in range(len(ListTBond)):
    row = ListTBond[i]
    record = list()
    record.append(row[0])
    record.append(DatetimeToDate(row[1]))
    record.append(DatetimeToDate(row[2]))
    record.append(row[3])
    record.append(row[4])
    record.append(row[5])
    TBondBook.append(record)

In [9]:
TBondBook

[['A10105', Date(21,5,2021), Date(21,5,2041), 20, 0.00625, 10000000],
 ['A10106', Date(18,6,2021), Date(18,6,2023), 2, 0.00125, 20000000],
 ['A10107', Date(23,6,2021), Date(23,6,2031), 10, 0.00375, 50000000]]

In [10]:
h_TSZero = ql.YieldTermStructureHandle(TSZero)
TBondEngine = ql.DiscountingBondEngine(h_TSZero)

In [11]:
ListZero = ListRate
VecDate = ql.DateVector(ListDate)

In [12]:
ListZero

[0.001861857532461489,
 0.00200599090568911,
 0.002160664083285364,
 0.00265646846120931,
 0.003074269586973849,
 0.004170292230914892,
 0.00814077379984233]

In [13]:
NoPoint = len(ListZero)
print(NoPoint)
ListRW = list([RW, RW, RW, RW, RW, RW])

7


In [14]:
ListListZero = list()
ListListZero.append(ListZero)
for i in range(1, NoPoint-1):
    NewList = list()
    NewList.append(ListZero[0])
    for j in range(1, NoPoint):
        if(i==j):
            NewList.append(ListZero[j] + 0.0001)
        else:
            NewList.append(ListZero[j])
    ListListZero.append(NewList)

In [15]:
ListListZero[0]

[0.001861857532461489,
 0.00200599090568911,
 0.002160664083285364,
 0.00265646846120931,
 0.003074269586973849,
 0.004170292230914892,
 0.00814077379984233]

In [16]:
ListListZero[1]

[0.001861857532461489,
 0.00210599090568911,
 0.002160664083285364,
 0.00265646846120931,
 0.003074269586973849,
 0.004170292230914892,
 0.00814077379984233]

In [17]:
ListListZero[2]

[0.001861857532461489,
 0.00200599090568911,
 0.0022606640832853638,
 0.00265646846120931,
 0.003074269586973849,
 0.004170292230914892,
 0.00814077379984233]

In [18]:
ListListZero[5]

[0.001861857532461489,
 0.00200599090568911,
 0.002160664083285364,
 0.00265646846120931,
 0.003074269586973849,
 0.004270292230914892,
 0.00814077379984233]

In [19]:
NoCurve = NoPoint - 1
ListVecZero = list()
ListTSZero = list()
for i in range(NoCurve):
    AVecZero = ql.DoubleVector(ListListZero[i])
    ListVecZero.append(AVecZero)
    ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
    ListTSZero.append(ATSZero)

In [20]:
CurvUpListListZero = list()
CurvUpListListZero.append(ListZero)
for i in range(1, NoCurve):
    NewList = list()
    NewList.append(ListZero[0])
    for j in range(1, NoPoint):
        if(i==j):
            NewList.append(ListZero[j] + RW)
        else:
            NewList.append(ListZero[j])
    CurvUpListListZero.append(NewList)

In [21]:
CurvUpListListZero[0]

[0.001861857532461489,
 0.00200599090568911,
 0.002160664083285364,
 0.00265646846120931,
 0.003074269586973849,
 0.004170292230914892,
 0.00814077379984233]

In [22]:
CurvUpListListZero[1]

[0.001861857532461489,
 0.007005990905689111,
 0.002160664083285364,
 0.00265646846120931,
 0.003074269586973849,
 0.004170292230914892,
 0.00814077379984233]

In [23]:
CurvUpListListZero[2]

[0.001861857532461489,
 0.00200599090568911,
 0.007160664083285364,
 0.00265646846120931,
 0.003074269586973849,
 0.004170292230914892,
 0.00814077379984233]

In [24]:
CurvUpListListZero[5]

[0.001861857532461489,
 0.00200599090568911,
 0.002160664083285364,
 0.00265646846120931,
 0.003074269586973849,
 0.009170292230914891,
 0.00814077379984233]

In [25]:
CurvDnListListZero = list()
CurvDnListListZero.append(ListZero)
for i in range(1, NoCurve):
    NewList = list()
    NewList.append(ListZero[0])
    for j in range(1, NoPoint):
        if(i==j):
            NewList.append(ListZero[j] - RW)
        else:
            NewList.append(ListZero[j])
    CurvDnListListZero.append(NewList)

In [26]:
CurvDnListListZero[0]

[0.001861857532461489,
 0.00200599090568911,
 0.002160664083285364,
 0.00265646846120931,
 0.003074269586973849,
 0.004170292230914892,
 0.00814077379984233]

In [27]:
CurvDnListListZero[1]

[0.001861857532461489,
 -0.00299400909431089,
 0.002160664083285364,
 0.00265646846120931,
 0.003074269586973849,
 0.004170292230914892,
 0.00814077379984233]

In [28]:
CurvDnListListZero[2]

[0.001861857532461489,
 0.00200599090568911,
 -0.002839335916714636,
 0.00265646846120931,
 0.003074269586973849,
 0.004170292230914892,
 0.00814077379984233]

In [29]:
CurvDnListListZero[5]

[0.001861857532461489,
 0.00200599090568911,
 0.002160664083285364,
 0.00265646846120931,
 0.003074269586973849,
 -0.0008297077690851079,
 0.00814077379984233]

In [30]:
SenSheet = PstBook['CSR']
for i in range(0, len(TBondBook)):
    ListValue = list()

    record = TBondBook[i]
    startDate = record[1]
    maturityDate = record[2]
    tenor = record[3]
    coupon = record[4]
    faceAmt = record[5]
    DGR = ql.DateGeneration.Backward
    sch = ql.Schedule(startDate, maturityDate, ql.Period(ql.Annual), Cal, BDC, BDC, DGR, False)
    ListCoupon = list()
    for j in range(tenor):
        ListCoupon.append(coupon)
    VecCoupon = ql.DoubleVector(ListCoupon)
    TBond = ql.FixedRateBond(settlementDays, faceAmt, sch, VecCoupon, DC365, BDC, faceAmt)
    
    h_TS = ql.YieldTermStructureHandle(ListTSZero[0])
    TBondEngine = ql.DiscountingBondEngine(h_TS)
    TBond.setPricingEngine(TBondEngine)
    Value = TBond.cleanPrice()
    ListValue.append(Value)
    SenSheet.cell(i+5, 1).value = record[0]
    SenSheet.cell(i+5, 2).value = ListValue[0]
    
    for j in range(1, NoCurve):
        h_TS = ql.YieldTermStructureHandle(ListTSZero[j])
        TBondEngine = ql.DiscountingBondEngine(h_TS)
        TBond.setPricingEngine(TBondEngine)
        Value = TBond.cleanPrice()
        ListValue.append(Value)
    
    ListSensitivity = list()
    for j in range(1, len(ListValue)):
        ListSensitivity.append((ListValue[j] - ListValue[0])*10000)
    
    ListWS = list()
    for j in range(0, len(ListSensitivity)):
        ListWS.append(ListRW[j] * ListSensitivity[j])
        
    for j in range(0, len(ListSensitivity)):    
        SenSheet.cell(i+5, j+5).value = ListWS[j]
    
    ListVecZero = list()
    CurvUpListTSZero = list()
    for j in range(NoCurve):
        AVecZero = ql.DoubleVector(CurvUpListListZero[j])
        ListVecZero.append(AVecZero)
        ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
        CurvUpListTSZero.append(ATSZero)
                
    ListVecZero = list()
    CurvDnListTSZero = list()
    for j in range(NoCurve):
        AVecZero = ql.DoubleVector(CurvDnListListZero[j])
        ListVecZero.append(AVecZero)
        ATSZero = ql.ZeroCurve(VecDate, AVecZero, DC365, Cal, ql.Linear(), ql.Continuous, ql.Annual)
        CurvDnListTSZero.append(ATSZero)
        
    CurvUpListHTSZero = list()
    for j in range(NoCurve):
        CurvUpListHTSZero.append(ql.YieldTermStructureHandle(CurvUpListTSZero[j]))
        
    CurvDnListHTSZero = list()
    for j in range(NoCurve):
        CurvDnListHTSZero.append(ql.YieldTermStructureHandle(CurvDnListTSZero[j]))
        
    CurvUpValue = list()
    for j in range(NoCurve):
        TBondEngine = ql.DiscountingBondEngine(CurvUpListHTSZero[j])
        TBond.setPricingEngine(TBondEngine)
        Value = TBond.cleanPrice()
        CurvUpValue.append(Value)
    
    CurvDnValue = list()
    for j in range(NoCurve):
        TBondEngine = ql.DiscountingBondEngine(CurvDnListHTSZero[j])
        TBond.setPricingEngine(TBondEngine)
        Value = TBond.cleanPrice()
        CurvDnValue.append(Value)
        
    CurvatureUp = list()
    ShiftUpValue = list()
    DeltaChange = list()
    for j in range(1, NoCurve):
        ShiftUpValue.append(CurvUpValue[j]-CurvUpValue[0])        
        CurvatureUp.append(-((CurvUpValue[j]-CurvUpValue[0]) - RW * ListSensitivity[j-1]))
                
    for j in range(0, len(CurvatureUp)):    
        SenSheet.cell(i+5, j+12).value = CurvatureUp[j]
            
    CurvatureDn = list()
    ShiftDnValue = list()
    for j in range(1, NoCurve):
        ShiftDnValue.append(CurvDnValue[j]-CurvDnValue[0])    
        CurvatureDn.append(-((CurvDnValue[j]-CurvDnValue[0]) + RW * ListSensitivity[j-1]))

    for j in range(0, len(CurvatureDn)):    
        SenSheet.cell(i+5, j+19).value = CurvatureDn[j]
        
PstBook.save('Pst_TBond.xlsx')
PstBook.close()

In [31]:
ListValue

[47962489.931523226,
 47962489.93152181,
 47962489.93144915,
 47962489.931299776,
 47962306.31802058,
 47914800.48415786]